In [5]:
from datasets import load_dataset, Dataset
import sklearn

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
import numpy as np

import transformers

from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import evaluate
import time
import pandas as pd

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [7]:
df = pd.read_csv("IMDB Dataset.csv")

ds = Dataset.from_pandas(df)
print(ds)

print(ds)          # shows train/test splits
print(ds[0])   # inspect first example

full = ds  #full data set

print("Dataset size:", len(full))
print("Columns:", full.column_names)
print("First row:", full[0])

Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})
Dataset({
    features: ['review', 'sentiment'],
    num_rows: 50000
})
{'review': "One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and mor

In [8]:
# 2. Extract numpy arrays of text + label
reviews = np.array(full["review"])
sentiment = np.array(full["sentiment"])   # usually "positive"/"negative"

# Convert label strings into integers: positive → 1, negative → 0
label_map = {"positive": 1, "negative": 0}
labels = np.array([label_map[x] for x in sentiment])

In [9]:
# 3. First split: Train vs Holdout (val+test)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)#20% used for testing, 80% used for trainint, 
#setting a specific integer seed, the random shuffling process will be the same every time the code runs, meaning the exact same data points will end up in the training and testing sets across different executions
train_idx, hold_idx = next(sss.split(reviews, labels))

In [ ]:
# 4. Second split: Holdout → Validation + Test (50/50 split)
hold_reviews = reviews[hold_idx]
hold_labels = labels[hold_idx]
val_idx, test_idx = train_test_split(
    np.arange(len(hold_reviews)),
    test_size=0.5,
    stratify=hold_labels,
    random_state=50
) #split the hold dataset: half used for testing and half used for validation

In [12]:
# Convert back to dataset (HuggingFace)
train_ds = Dataset.from_dict({
    "review": reviews[train_idx],
    "labels": labels[train_idx]
})

val_ds = Dataset.from_dict({
    "review": hold_reviews[val_idx],
    "labels": hold_labels[val_idx]
})

test_ds = Dataset.from_dict({
    "review": hold_reviews[test_idx],
    "labels": hold_labels[test_idx]
})

print("Split sizes:")
print(" Train:", len(train_ds))
print(" Val:  ", len(val_ds))
print(" Test: ", len(test_ds))

Split sizes:
 Train: 40000
 Val:   5000
 Test:  5000


In [14]:
#%%Fine-Tuning DistilBERT

# 1) choose tokenizer and settings
MODEL_NAME = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

RemoteEntryNotFoundError: 404 Client Error. (Request ID: Root=1-691cf398-13805852284785e07de8893f;e838c529-c954-44b5-a755-055cfd56c2ea)

Entry Not Found for url: https://huggingface.co/api/models/distilbert/distilbert-base-uncased/tree/main/additional_chat_templates?recursive=false&expand=false.
additional_chat_templates does not exist on "main"